In [21]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications import VGG16
from keras.optimizers import Adam
import splitfolders

In [30]:
train_dir = 'train'
val_dir = 'val'
test_dir = 'test'

BATCH_SIZE = 64
IMG_SIZE = (224, 224)

nb_train_samples = 320 * 2
nb_validation_samples = 40 * 2
nb_test_samples = 40 * 2

path = "augmented_images"

In [31]:
input_folder = path
output = "images_for_training"
splitfolders.ratio(input_folder, output=output, seed=42, ratio=(.8, .1, .1))

Copying files: 800 files [00:01, 576.92 files/s]


In [32]:
vgg16_net = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

vgg16_net.trainable = False
vgg16_net.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [33]:
model = Sequential()
model.add(vgg16_net)
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(2, activation="softmax"))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_4 (Flatten)         (None, 25088)             0         
                                                                 
 dense_8 (Dense)             (None, 256)               6422784   
                                                                 
 activation_4 (Activation)   (None, 256)               0         
                                                                 
 dropout_4 (Dropout)         (None, 256)               0         
                                                                 
 dense_9 (Dense)             (None, 2)                 514       
                                                                 
Total params: 21,137,986
Trainable params: 6,423,298
N

In [34]:
model.compile(loss = 'categorical_crossentropy', optimizer=Adam(learning_rate=1e-5), metrics=['accuracy'])

In [35]:
datagen = ImageDataGenerator(rescale=1.0/255)

In [36]:
train_generator = datagen.flow_from_directory(
    "images_for_training/train",
    target_size=(224, 224), batch_size=BATCH_SIZE, class_mode='categorical')

val_generator = datagen.flow_from_directory(
    "images_for_training/val",
    target_size=(224, 224), batch_size=BATCH_SIZE, class_mode='categorical')

test_generator = datagen.flow_from_directory(
    "images_for_training/test",
    target_size=(224, 224), batch_size=BATCH_SIZE, class_mode='categorical')

Found 640 images belonging to 2 classes.
Found 80 images belonging to 2 classes.
Found 80 images belonging to 2 classes.


In [28]:
model.fit(
    train_generator, steps_per_epoch=nb_train_samples // BATCH_SIZE, epochs=5,
    validation_data=val_generator, validation_steps=nb_validation_samples // BATCH_SIZE)

Epoch 1/5
10/10 [==============================] - 202s 20s/step - loss: 0.7910 - accuracy: 0.4891 - val_loss: 0.6124 - val_accuracy: 0.6406
Epoch 2/5
10/10 [==============================] - 191s 19s/step - los

In [29]:
scores = model.evaluate_generator(test_generator, nb_test_samples // BATCH_SIZE)
print("accuracy: %.2f%%" % (scores[1]*100))


C:\Users\Huawei-PC\AppData\Local\Temp\ipykernel_12132\2619268717.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  scores = model.evaluate_generator(test_generator, nb_test_samples // BATCH_SIZE)


accuracy: 89.06%
